# Running vsn

First, generate a metadata file as described here: https://vsn-pipelines.readthedocs.io/en/latest/scatac-seq.html

You can do this manually, but I did it automatically using some python string manipulations (see notebook 1b)

In [1]:
cat metadata.tsv

sample_name	technology	fastq_PE1_path	fastq_barcode_path	fastq_PE2_path
TXG_10xmultiome_e18mousebrainfresh.FULL	multiome_revcomp	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xmultiome_e18mousebrainfresh__R1.FULL.fastq.gz	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xmultiome_e18mousebrainfresh__R2.FULL.fastq.gz	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xmultiome_e18mousebrainfresh__R3.FULL.fastq.gz
TXG_10xv11_adultmousecortexchromiumx.FULL	atac_revcomp	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xv11_adultmousecortexchromiumx__R1.FULL.fastq.gz	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xv11_adultmousecortexchromiumx__R2.FULL.fastq.gz	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xv11_adultmousecortexchromiumx__R3.FULL.fastq.gz
TXG_10xv1_adultmousefresh.FULL	atac_revcomp	../1_data_repository/publicdata_full_fastq/all_merged/TXG_10xv1_adultmousefresh__R1.FULL.fastq.gz	../1_data_repository/publi

Generate a config file read by the vsn pipeline. $nwork is a temporary file destination.

In [3]:
VSN=../0_resources/vsn-pipelines/main_atac.nf

In [17]:
nwork=${VSC_SCRATCH}/ATAC_BENCHMARK_FULL/
mkdir $nwork
export NXF_WORK=$nwork


../0_resources/nextflow/nextflow-21.04.3-all config $VSN \
    -profile atac_preprocess_bap,vsc \
    > atac_preprocess.config

mkdir: cannot create directory ‘/scratch/leuven/330/vsc33042/ATAC_BENCHMARK_FULL/’: File exists


### now make some changes to the config:
* redirect to correct metadata_auto.tsv file
* fix BWA parameters
    * change the bwa index directory to the right, in this case hg39
    * change the bwa executor to local to run on current node
    * number of bwa CPUs: better to have 2 forks running with 17 threads than to have 1 fork with 36 threads due to I/O overhead
* add whitelists for each sample
* check if bap parameters are correct
* make sure all output dirs etc. exist
* Increase the time limit

Here is a functional config file tailored to our computing environment:

In [4]:
cat atac_preprocess_rapid.config

manifest {
   name = 'vib-singlecell-nf/vsn-pipelines'
   description = 'A repository of pipelines for single-cell data in Nextflow DSL2'
   homePage = 'https://github.com/vib-singlecell-nf/vsn-pipelines'
   version = '0.27.0'
   mainScript = 'main.nf'
   defaultBranch = 'master'
   nextflowVersion = '!21.04.3'
}

params {
   global {
      project_name = 'benchmark_atac'
      outdir = 'public_preprocessing_output'
   }
   misc {
      test {
         enabled = false
      }
   }
   utils {
      container = 'vibsinglecellnf/utils:0.4.0'
      file_converter {
         off = 'h5ad'
         tagCellWithSampleId = true
         remove10xGEMWell = false
         useFilteredMatrix = true
         makeVarIndexUnique = false
      }
      publish {
         compressionLevel = 6
         annotateWithBatchVariableName = false
         mode = 'copy'
      }
   }
   atac_preprocess_tools {
      mark_duplicates_method = 'MarkDuplicates'
      adapter_trimming_method = 'Trim_Galore'
   }
   data

I suggest you run a difftool on this to compare this to your config.

Then, in a tmux session to avoid interruption:

In [2]:
sleep 2h

In [ ]:
VSN=../0_resources/vsn-pipelines_/main_atac.nf
NXF=/readonly/dodrio/scratch/projects/starting_2022_023/software/nextflow/nextflow-21.04.3/nextflow
$NXF -C atac_preprocess_rapid.config run $VSN -entry atac_preprocess_rapid

In [ ]:
commented some shit 